# Credit Card Fraud Prediction - Loading Dataset using Snowpark Python

This example is based on the Machine Learning for Credit Card Fraud detection - Practical handbook, https://fraud-detection-handbook.github.io/fraud-detection-handbook/

## Loading Credit Card Transactions into Snowflake

### Import the dependencies and connect to Snowflake

In [5]:
# Snowpark
import snowflake.snowpark as snp
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Session
import json

print(f"Using Snowpark for Python version: {snp.__version__}")

Using Snowpark for Python version: 1.0.0


Get connection parameters

In [6]:
#Create ses
f = open('/Users/erich.wiehahn/Documents/Training/Snowflake/params.json') #insert creds here
params = json.load(f)

connection_parameters = {
    "user": params['user'],
    "account": params['account'],
    "password": params['password'],
    "database": "SNOWFLAKE",
    "schema": "PUBLIC",
    "role": "ACCOUNTADMIN"
}
session = Session.builder.configs(connection_parameters).create()
#session.sql_simplifier_enabled = True
print(f"Current schema: {session.get_fully_qualified_current_schema()}")
print(f"Current role: {session.get_current_role()}")
print(f"Current warehouse: {session.get_current_warehouse()}")

Current schema: "SNOWFLAKE"."PUBLIC"
Current role: "ACCOUNTADMIN"
Current warehouse: "COMPUTE_WH"


In [8]:
#Create database and schema if not already existing
session.sql("CREATE DATABASE IF NOT EXISTS credit_fraud").collect()
session.sql("USE DATABASE credit_fraud").collect()
print(f"Current schema: {session.get_fully_qualified_current_schema()}")

Current schema: "CREDIT_FRAUD"."PUBLIC"


### Define Staging Area and the Schema for the transaction table

Using SQL we can create a internal stage and then use the **put** function to uplad the **fraud_transactions.csv.gz** file to it.

In [9]:
stage_name = "FRAUD_DATA"
# Create a internal staging area for uploading the source file
session.sql(f"CREATE or replace STAGE {stage_name}").collect()

# Upload the source file to the stage
putResult = session.file.put("data/fraud_transactions.csv.gz", f"@{stage_name}", auto_compress=False, overwrite=True)

putResult

[PutResult(source='fraud_transactions.csv.gz', target='fraud_transactions.csv.gz', source_size=21382572, target_size=21382576, source_compression='GZIP', target_compression='GZIP', status='UPLOADED', message='')]

Define the schma for our **CUSTOMER_TRANSACTIONS_FRAUD** table

In [10]:
# Define the schema for the Frauds table
dfCustTrxFraudSchema = T.StructType(
    [
        T.StructField("TRANSACTION_ID", T.IntegerType()),
        T.StructField("TX_DATETIME", T.TimestampType()),
        T.StructField("CUSTOMER_ID", T.IntegerType()),
        T.StructField("TERMINAL_ID", T.IntegerType()),
        T.StructField("TX_AMOUNT", T.FloatType()),
        T.StructField("TX_TIME_SECONDS", T.IntegerType()),
        T.StructField("TX_TIME_DAYS", T.IntegerType()),
        T.StructField("TX_FRAUD", T.IntegerType()),
        T.StructField("TX_FRAUD_SCENARIO", T.IntegerType())
    ]
)

Load the **fraud_transactions.csv.gz** to a DataFrame reader and save into a table

In [11]:
# Crete a reader
dfReader = session.read.schema(dfCustTrxFraudSchema)

# Get the data into the data frame
dfCustTrxFraudRd = dfReader.csv(f"@{stage_name}/fraud_transactions.csv.gz")

In [12]:
# Write the dataframe in a table
ret = dfCustTrxFraudRd.write.mode("overwrite").saveAsTable("CUSTOMER_TRANSACTIONS_FRAUD")

In [13]:
#We can now query this table
session.sql("SELECT * FROM CUSTOMER_TRANSACTIONS_FRAUD").show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TRANSACTION_ID"  |"TX_DATETIME"        |"CUSTOMER_ID"  |"TERMINAL_ID"  |"TX_AMOUNT"  |"TX_TIME_SECONDS"  |"TX_TIME_DAYS"  |"TX_FRAUD"  |"TX_FRAUD_SCENARIO"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------
|0                 |2019-04-01 00:00:31  |596            |3156           |57.16        |31                 |0               |0           |0                    |
|1                 |2019-04-01 00:02:10  |4961           |3412           |81.51        |130                |0               |0           |0                    |
|2                 |2019-04-01 00:07:56  |2              |1365           |146.0        |476                |0               |0           |0                    |
|3                 |2019-04-01 00:

### Read the data from the staging area and create CUSTOMER_TRANSACTIONS_FRAUD, CUSTOMERS and TERMINALS tables

In [14]:
# Now create Customers and Terminal tables

dfCustTrxFraudTb =session.table("CUSTOMER_TRANSACTIONS_FRAUD")

dfCustomers = dfCustTrxFraudTb.select(F.col("CUSTOMER_ID")).distinct().sort(F.col("CUSTOMER_ID"))

dfTerminals = dfCustTrxFraudTb.select(F.col("TERMINAL_ID")).distinct().sort(F.col("TERMINAL_ID"))
                                
ret2 = dfCustomers.write.mode("overwrite").saveAsTable("CUSTOMERS")

ret3 = dfTerminals.write.mode("overwrite").saveAsTable("TERMINALS")

In [15]:
dfCustTrxFraudTb.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TRANSACTION_ID"  |"TX_DATETIME"        |"CUSTOMER_ID"  |"TERMINAL_ID"  |"TX_AMOUNT"  |"TX_TIME_SECONDS"  |"TX_TIME_DAYS"  |"TX_FRAUD"  |"TX_FRAUD_SCENARIO"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------
|0                 |2019-04-01 00:00:31  |596            |3156           |57.16        |31                 |0               |0           |0                    |
|1                 |2019-04-01 00:02:10  |4961           |3412           |81.51        |130                |0               |0           |0                    |
|2                 |2019-04-01 00:07:56  |2              |1365           |146.0        |476                |0               |0           |0                    |
|3                 |2019-04-01 00:

In [16]:
pandas_sample = session.sql("SELECT * FROM CUSTOMER_TRANSACTIONS_FRAUD LIMIT 10").toPandas()
pandas_sample

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2019-04-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2019-04-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2019-04-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2019-04-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2019-04-01 00:10:34,927,9906,50.99,634,0,0,0
5,5,2019-04-01 00:10:45,568,8803,44.71,645,0,0,0
6,6,2019-04-01 00:11:30,2803,5490,96.03,690,0,0,0
7,7,2019-04-01 00:11:44,4684,2486,24.36,704,0,0,0
8,8,2019-04-01 00:11:53,4128,8354,26.34,713,0,0,0
9,9,2019-04-01 00:13:44,541,6212,59.07,824,0,0,0


In [17]:
session.close()

In [18]:
#session.cancel_all()